# EDA

- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [391]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json

## Data Importingm

In [392]:
# Function return DataFrame from JSON file
def json_to_df(file):
    # Load the JSON data
    with open(file, 'r') as f:
        data = json.load(f)

    # pprint(data['data']['results'][0])
    #.explode().apply(pd.Series)

    # Convert JSON data to a DataFrame
    return pd.json_normalize(data['data']['results'])

In [393]:
# load one file first to see what type of data you're dealing with and what attributes it has
tmp_df = pd.read_json('../data/housing/AK_Juneau_0.json')
# tmp_df['data']['results']

In [394]:
# Display the DataFrame to check the data and its attributes
# tmp_df = json_to_df('../data/housing/AK_Juneau_0.json')
# tmp_df.head()

In [395]:
# Create empty DataFrame for housing
df = pd.DataFrame()
# The path to the data files
dir_path = '../data/housing/'

# loop over all files and put them into a dataframe
# List all files in the specified directory

n = 0
for file_name in sorted(os.listdir(dir_path)):
    n += 1
    if file_name.endswith('.json'):
        # print(f'File: {file_name}')
        file_path = os.path.join(dir_path, file_name)
        tmp_df = json_to_df(file_path)
        tmp_df = tmp_df.dropna(axis=1, how='all')
        df = pd.concat([df, tmp_df], ignore_index=True)

    # if n > 1: break
df

,last_update_date,tags,permalink,status,list_date,branding,list_price,property_id,photos,listing_id,...,virtual_tours,description.sold_price,description.baths_half,location.county.fips_code,flags.is_foreclosure,description.baths_3qtr,source.plan_id,source.spec_id,community.advertisers,community.description.name
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",2957241843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9424983842,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9479068516,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9879331943,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9521639574,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaningm and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [396]:
df.shape

(8191, 50)

In [397]:
# load and concatenate data here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   branding                               8159 non-null   object 
 6   list_price                             7721 non-null   float64
 7   property_id                            8159 non-null   object 
 8   photos                                 7403 non-null   object 
 9   listing_id                             7752 non-null   object 
 10  price_reduced_amount                   2484 non-null   float64
 11  matt

In [398]:
columns = df.columns.sort_values().to_list()
# columns

In [399]:
# Rearrange DataFrame columns
df = df[columns]

In [400]:
# drop or replace values as necessary
df = df.drop(
    columns=[
        'branding',
        'community.advertisers',
        'community.description.name',
        # 'description.baths',
        # 'description.baths_3qtr',
        # 'description.baths_full',
        # 'description.baths_half',
        # 'description.beds',
        # 'description.garage',
        # 'description.lot_sqft',
        # 'description.sold_date',
        # 'description.sold_price',
        # 'description.sqft',
        # 'description.stories',
        'description.sub_type',
        'description.type',
        # 'description.year_built',
        # 'flags.is_foreclosure',
        # 'flags.is_new_listing',
        # 'flags.is_price_reduced',
        'last_update_date',
        'lead_attributes.show_contact_an_agent',
        # 'list_date',
        # 'list_price',
        'listing_id',
        # 'location.address.city',
        # 'location.address.coordinate.lat',
        # 'location.address.coordinate.lon',
        'location.address.line',
        # 'location.address.postal_code',
        # 'location.address.state',
        # 'location.address.state_code',
        # 'location.county.fips_code',
        # 'location.county.name',
        'location.street_view_url',
        # 'matterport',
        'other_listings.rdc',
        'permalink',
        'photos',
        # 'price_reduced_amount',
        'primary_photo.href',
        # 'products.brand_name',
        # 'property_id',
        'source.agents',
        'source.plan_id',
        'source.spec_id',
        # 'source.type',
        # 'status',
        # 'tags',
        'virtual_tours'
    ],
    errors='ignore'
)

In [401]:
df

,description.baths,description.baths_3qtr,description.baths_full,description.baths_half,description.beds,description.garage,description.lot_sqft,description.sold_date,description.sold_price,description.sqft,...,location.address.state_code,location.county.fips_code,location.county.name,matterport,price_reduced_amount,products.brand_name,property_id,source.type,status,tags
0,2.0,NaN,2.0,NaN,3.0,1.0,10454.0,2023-09-18,NaN,1821.0,...,AK,NaN,Juneau,False,45000.0,basic_opt_in,9074430767,mls,sold,"[carport, community_outdoor_space, cul_de_sac,..."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,...,AK,NaN,Juneau,False,NaN,NaN,9424983842,NaN,sold,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,...,AK,NaN,Juneau,False,NaN,NaN,9479068516,NaN,sold,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,...,AK,NaN,Juneau,False,NaN,NaN,9879331943,NaN,sold,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,...,AK,NaN,Juneau,False,NaN,NaN,9521639574,NaN,sold,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [408]:
columns = {col: col.split('.')[-1] for col in df.columns}
# columns
df.rename(columns=columns, inplace=True)
df = df[sorted(df.columns)]
# df.head()


### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [403]:
# OHE categorical variables / tags here
# tags will have to be done manually

In [409]:
# One-Hot Encode the 'tags', 'city', and 'state' columns, considering low frequency removal for 'tags'
ohe_df = df.copy()
tag_threshold = 10  # Minimum frequency for 'tags' to be included
tags = ohe_df['tags'].explode().value_counts()
common_tags = tags[tags >= tag_threshold].index
ohe_df['tags'] = ohe_df['tags'].apply(lambda x: [tag for tag in x if tag in common_tags] if isinstance(x, list) else x)

# Get dummy variables
dummies_tags = ohe_df['tags'].str.join('|').str.get_dummies()
dummies_city = pd.get_dummies(ohe_df['city'], prefix='city')
dummies_state = pd.get_dummies(ohe_df['state'], prefix='state')

ohe_df = pd.concat([ohe_df, dummies_tags, dummies_city, dummies_state], axis=1).drop(['tags', 'city', 'state'], axis=1)
ohe_df.shape

(8191, 322)

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [405]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [406]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [407]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized